# AdjSim Tutorial

AdjSim is an agent-based modelling engine. It allows users to define simulation environments through which agents interact through ability casting and timestep iteration. It is tailored towards allowing agents to behave intelligently through the employment of Reinforcement Learning techniques such as Q-Learning. 

This tutorial will enumerate many of AdjSim's features through the construction of a simulation where a group of agents will play a game of tag. Lets dive in!

---

## The Simulation Space

AdjSim simulations take place inside the fittingly named `Simulation` object.

Lets begin by importing our libararies.

In [1]:
import adjsim
import numpy as np # AdjSim also heavily relies on numpy. Its usage is recommended.

We will make a trivial empty simulation to display its usage.

In [2]:
sim = adjsim.core.Simulation()

# Interactive simulation method.
sim.start() # Begin the simulation.
sim.step() # Take one step.
sim.step(10) # Take 10 steps.
sim.end() # end the simulation.

# Batch simulation method.
sim.simulate(100) # Start, Simulate for 100 timesteps, End. All in one go.

Simulating timestep 100/100 - population: 0

Its a little lonely in that simulation, lets add an agent.

In [3]:
sim.agents.add(adjsim.core.Agent())
sim.agents.add(adjsim.core.Agent())

sim.simulate(100)

Simulating timestep 100/100 - population: 2 

Things are happening, but its a little hard to see. Lets visualize our simulation space. Simulations and agents have a hierarchal class structure. `VisualSimulation` inherits from `Simulation`, and can be used to display a 2D simulation environment.

In [4]:
sim = adjsim.core.VisualSimulation()
sim.simulate(15)

Simulating timestep 15/15 - population: 0 

Similarly, an appropriate `VisualAgent` must be present in the simulation space if it is to be seen. The agent inheritance hierarchy is as follows:

1. `Agent`: The base agent class. Contains the attributes that are minimally needed in order for an agent to be simulated.
2. `SpatialAgent`: Adds the presence of the `pos` Attribute, allowing for a 2D position to be associated with an agent.
3. `VisualAgent`: Adds the attributes needed for visualization on top of those of `SpatialAgent`. This is the minimum class needed for an agent to appear inside a `VisualSimulation`.

In [5]:
sim.agents.add(adjsim.core.VisualAgent(pos=np.array([1,1])))
sim.simulate(15)

Simulating timestep 15/15 - population: 1 

Now we should see our agent! Next, lets make it do things.

---

## Agents

Agents are simulation artifacts that manipulate the simulation environment. In the adjsim model, within each timestep agents take turns acting out their environment manipulations. One iteration of an agent's environment manipulations is known as an agent's __step__.

There are two major aspects to an agent's __step__: 

1. A set of __actions__. An __action__ defines one distinct user-defined set of computations that the agent manipulates its environment with. It is simply a python function. The actions we will be defining in our simulation of the game of tag will be a _move_ and a _tag_ action.
2. A decision module. Decision modules are AdjSim objects that an agent uses to choose which actions to perform, and in what order.

We'll start by making an agent that can _move_, but that can't _tag_. The _move_ action will allow an agent to move in a random direction bounded by a square arena.

In [6]:
# Constants.
ARENA_BOUND = 100
MOVE_DIST = 20

# Actions must follos the following signiature. 
# The Simulation object will be passed in as the 'simulation' parameter.
# The agent calling the action will be passed as the 'source' parameter.
def move(simulation, source):
    movement = (np.random.rand(2) - 0.5) * MOVE_DIST
    source.pos = np.clip(source.pos + movement, -ARENA_BOUND, ARENA_BOUND)
    source.step_complete = True

The final line, `source.step_complete = True`, lets the decision module know that no further actions can be completed in the agent's __step__ after the current one. 

Now let's create an agent that uses this action. The agent we will be creating will have a `RandomRepeatedCastDecision`. This decision module will randomly select actions and invoke them until the `source.step_complete` attribute is true. This should result in one cast of the above-defined _move_ action.

In [7]:
class Mover(adjsim.core.VisualAgent):
    def __init__(self, x, y):
        super().__init__(pos=np.array([x, y]))
        
        # Set the decision module.
        self.decision = adjsim.decision.RandomSingleCastDecision()

        # Populate the agent's action list.
        self.actions["move"] = move

Let's take it out for a spin.

In [8]:
sim = adjsim.core.VisualSimulation()
sim.agents.add(Mover(0, 0))

sim.simulate(15)

Simulating timestep 15/15 - population: 1 

We observe an agent moving in random directions in each timestep. Nice! Lets give it some friends.

In [9]:
class MoverSimulation(adjsim.core.VisualSimulation):
    def __init__(self):
        super().__init__()

        for i in range(5):
            for j in range(5):
                self.agents.add(Mover(20*i, 20*j))
                
sim = MoverSimulation()
sim.simulate(15)

Simulating timestep 15/15 - population: 25 

We should now see a group of 25 agents moving in a similar fashion to our first one. The basics are now covered. Lets put together our simulation of a game of tag.

---

## Putting It All Together

The following will describe our simulation of a game of tag.

In [10]:
# Reiterate imports.
import adjsim
import numpy as np
import sys

# Constants.
ARENA_BOUND = 100
TAG_DIST_SQUARE = 100
MOVE_DIST = 20

def move(simulation, source):
    movement = (np.random.rand(2) - 0.5) * MOVE_DIST
    source.pos = np.clip(source.pos + movement, -ARENA_BOUND, ARENA_BOUND)
    source.step_complete = True

def tag(simulation, source):  
    if not source.is_it:
        return

    # Find nearest neighbour.
    closest_distance = sys.float_info.max
    nearest_neighbour = None
    for agent in simulation.agents:
        if agent.id == source.id:
            continue

        distance = adjsim.utility.distance_square(agent, source)
        if distance < closest_distance:
            nearest_neighbour = agent
            closest_distance = distance

    if closest_distance > TAG_DIST_SQUARE:
        return

    assert nearest_neighbour

    # Perform Tag.
    nearest_neighbour.is_it = True
    nearest_neighbour.color = adjsim.color.RED_DARK # This will change the agent's visual color.
    nearest_neighbour.order = 1 # Order describes what order the agents will take their steps in the simulation loop.
    source.is_it = False
    source.order = 0 # Order describes what order the agents will take their steps in the simulation loop.
    source.color = adjsim.color.BLUE_DARK # This will change the agent's visual color.

class Tagger(adjsim.core.VisualAgent):

    def __init__(self, x, y, is_it):
        super().__init__()

        self.is_it = is_it
        self.color = adjsim.color.RED_DARK if is_it else adjsim.color.BLUE_DARK
        self.pos = np.array([x, y])
        self.decision = adjsim.decision.RandomSingleCastDecision()

        self.actions["move"] = move
        self.actions["tag"] = tag

        if is_it:
            self.order = 1 # Order describes what order the agents will take their steps in the simulation loop.


class TaggerSimulation(adjsim.core.VisualSimulation):

    def __init__(self):
        super().__init__()

        for i in range(5):
            for j in range(5):
                self.agents.add(Tagger(20*i, 20*j, False))

        self.agents.add(Tagger(10, 10, True))
        
sim = TaggerSimulation()
sim.simulate(15)

Simulating timestep 15/15 - population: 26 

And that's it! We should observe a game of tag being played where the 'it' agent is red. It will tag the agents around it when it has the chance. 

---

## Filling the Noggin

The agents so far act randomly. Let's fix that.

### Using Reinforcement Learning-Enabled Decision Models

In this part of the tutorial we will be exploring AdjSim's Functional Decision model. This is the architecture that is used by the decision modules that employ Reinforcement Learning within AdjSim. 

The model asserts that there are two additional pieces of information that need to be given to an agent in order for it to be able to make intelligent decisions:

1. A __Loss Function__: This function essentially gives an agent a score as to how well its doing. Functional decision modules attempy to __minimize__ their associated loss function. For example, taggers try to minimize the amount of time they spend tagged, or bacteria try to maximize their calories.
2. A __Perception Function__: A function that processes the simulation from the perspective of an agent, and returns an object that is then used as input to the decision module for the purposes of invoking intelligent actions. For example, a given `Tagger` should be aware of where the `Tagger` that is 'it' is located relative to itsef. The perception acts as a filter between the omniscient data present within a simulation and the data that an agent may use to make its decisions. An invocation of a __Perception__ at a particular timestep is known as a __Observation__.



Overall, the processing workflow in an agent __step__ is the following.

$$
\text{Simulation} \rightarrow \text{Perception Function} \rightarrow \text{(Observation)} \rightarrow \text{Decision Module}
$$

Then the decision module is responsible for selecting and invoking the appropriate actions.

$$
\text{Decision Module} \rightarrow \text{Actions} \rightarrow \text{(Environment Manipulations)}
$$

Finally, the decision module evaluates the efficacy of its actions through the __Loss Function__ evaluation. It uses these results to train itself to make better future decisions.

$$
\text{Decision Module} \rightarrow \text{Loss Function} \rightarrow \text{(Efficacy Evaluation Score)} \rightarrow \text{Decision Module}
$$



We're going to start by making the agents of our simulation choose the right action between moving and tagging. Note that the direction of movement is still random. Parameterized functions will be covered in the next section. For our __Loss Function__, we will be using the `is_it` attribute to discourage being tagged. 

In [11]:
# Call signiature must be the following, same as an action.
def tagger_loss(simulation, source):
    return 10 if source.is_it else 0

Our __Perception Function__ will return two parameters:

1. Whether or not the agent is 'it'
2. The relative location of the nearest agent if the agent is 'it', otherwise the relative location of the 'it' agent. This will be done in polar coordinates.

Since we will be using Q-Learning for this simulation, each __Observation__ will define a state of the agent. Since relative location involves continuous values (floats), we find ourselves with an infinite number of possible agent states. In order to make the algorithm palpable we will discretize the values obtained that would otherwise be returned from the perception function. This need to discretize states is a idiosyncracy of the Q-Learning algorithm, and may not be needed when other decision modules are used.

In [12]:
import math

def find_it_tagger(simulation, source):
    for agent in simulation.agents:
        if agent.is_it:
            return (agent, adjsim.utility.distance_square(agent, source))
        
    # Raise an error if not found, this should never happen.
    raise Exception("'it' agent not found.")

def find_closest_non_it_tagger(simulation, source):
    closest_distance = sys.float_info.max
    nearest_neighbour = None
    for agent in simulation.agents:
        if agent.id == source.id:
            continue

        distance = adjsim.utility.distance_square(agent, source)
        if distance < closest_distance:
            nearest_neighbour = agent
            closest_distance = distance

    return (nearest_neighbour, closest_distance)

def tagger_perception(simulation, source):
    distance = None
    agent = None
    
    # Find appropriate agent and distance.
    if source.is_it:
        agent, distance = find_closest_non_it_tagger(simulation, source)
    else:
        agent, distance = find_it_tagger(simulation, source)

    # Obtain theta value.
    delta = agent.pos - source.pos
    theta = math.atan(delta[1]/delta[0]) if delta[0] != 0 else np.sign(delta[1])*math.pi/2

    # Discretize observation to reduce number of possible states.
    rounded_distance = round(distance/40)*40
    rounded_theta = round(theta/5)*5

    return (rounded_distance, rounded_theta, source.is_it)

Now, let's put together our simulation. It is important to note that we will be sharing our decision module object across all our Taggers in the same way that actions are shared. This will allow training to occur collectively, and allow all agents to learn from each other's enterprises.

In [13]:
class SomewhatCleverTagger(adjsim.core.VisualAgent):

    def __init__(self, x, y, is_it, _decision):
        super().__init__()

        self.is_it = is_it
        self.color = adjsim.color.RED_DARK if is_it else adjsim.color.BLUE_DARK
        self.pos = np.array([x, y])
        self.decision = _decision

        self.actions["move"] = move
        self.actions["tag"] = tag

        if is_it:
            self.order = 1


class SomewhatCleverTaggerSimulation(adjsim.core.VisualSimulation):

    def __init__(self):
        super().__init__()
        
        # Let's create the collective decision module. We will load and save our progress to the same file.
        io_file_name = "somewhat_clever_tagger.qlearning.pkl"
        self.tagger_decision = adjsim.decision.QLearningDecision(perception=tagger_perception, loss=tagger_loss, 
            callbacks=self.callbacks, input_file_name=io_file_name, output_file_name=io_file_name)

        for i in range(5):
            for j in range(5):
                self.agents.add(SomewhatCleverTagger(20*i, 20*j, False, self.tagger_decision))

        self.agents.add(SomewhatCleverTagger(10, 10, True, self.tagger_decision))
        
sim = SomewhatCleverTaggerSimulation()
sim.simulate(15)

Q Learning training data found, loading from somewhat_clever_tagger.qlearning.pkl...done
Simulating timestep 15/15 - population: 26 Processing Q-Learning data...done
Saving Q-Learning data...done


So now we are prepared to run the simulation a series of times, and after training the bacteria will be able to choose _which_ of _move_ or _tag_ they should choose. We will go into more detail regarding the training/testing cycle in the next example, where the results will be more flashy.

---

### Parameterizing Actions Using Decision-Mutable Values

We not only want our taggers to know which action to perform (between _move_ and _tag_), but also what direction to move when they perform their _move_ action. We accomplish this using Decision-Mutable Values. These are variable that a decision module will try to optimize for when performing its training cycle. These values are never explicitly set by the user. The decision module will set its value during runtime, before calling each action. These values are effectively read-only for the consumer of the API. 

Let's take a look. We will begin by re-writing our _move_ function.

In [17]:
def move(simulation, source):
    # We need to convert polar movement coordinates to cartesian.
    move_rho = source.move_rho.value
    move_theta = source.move_theta.value

    dx = math.cos(move_theta) * move_rho
    dy = math.sin(move_theta) * move_rho

    movement = np.array([dx, dy])

    source.pos = source.pos + movement
    source.step_complete = True
        
        
class ProperlyCleverTagger(adjsim.core.VisualAgent):
    def __init__(self, x, y, is_it, _decision):
        super().__init__()

        self.is_it = is_it
        self.color = adjsim.color.RED_DARK if is_it else adjsim.color.BLUE_DARK
        self.pos = np.array([x, y])
        self.decision = _decision
        
        # This is where the new magic lies. 
        self.move_rho = adjsim.decision.DecisionMutableFloat(0, MOVE_DIST)
        self.move_theta = adjsim.decision.DecisionMutableFloat(0, 360)

        self.actions["move"] = move
        self.actions["tag"] = tag

        if is_it:
            self.order = 1

Now we have properly clever taggers. Let's train them. We will make a train simulation and a test simulation, and run the training for a number of epochs. Let's see how they do!

In [18]:
class ProperlyCleverTaggerTrainSimulation(adjsim.core.Simulation): # Note that the simulation is not visual for training.

    def __init__(self):
        super().__init__()
        
        io_file_name = "properly_clever_tagger.qlearning.pkl"
        self.tagger_decision = adjsim.decision.QLearningDecision(perception=tagger_perception, loss=tagger_loss, 
            callbacks=self.callbacks, input_file_name=io_file_name, output_file_name=io_file_name)

        for i in range(5):
            for j in range(5):
                self.agents.add(ProperlyCleverTagger(20*i, 20*j, False, self.tagger_decision))

        self.agents.add(ProperlyCleverTagger(10, 10, True, self.tagger_decision))
        
class ProperlyCleverTaggerTestSimulation(adjsim.core.VisualSimulation): # Note that the simulation is not visual for training.

    def __init__(self):
        super().__init__()
        
        io_file_name = "properly_clever_tagger.qlearning.pkl"
        self.tagger_decision = adjsim.decision.QLearningDecision(perception=tagger_perception, loss=tagger_loss, 
            callbacks=self.callbacks, input_file_name=io_file_name, output_file_name=io_file_name, nonconformity_probability=0)

        for i in range(5):
            for j in range(5):
                self.agents.add(ProperlyCleverTagger(20*i, 20*j, False, self.tagger_decision))

        self.agents.add(ProperlyCleverTagger(10, 10, True, self.tagger_decision))
        
sim = ProperlyCleverTaggerTestSimulation()
sim.simulate(50)

# Training cycle.
EPOCHS = 10
for i in range(EPOCHS):
    sim = ProperlyCleverTaggerTrainSimulation()
    sim.simulate(300)
    
# Testing Cycle.
sim = ProperlyCleverTaggerTestSimulation()
sim.simulate(50)

Q Learning training data found, loading from somewhat_clever_tagger.qlearning.pkl...done
Simulating timestep 300/300 - population: 26 Processing Q-Learning data...done
Saving Q-Learning data...done
Q Learning training data found, loading from somewhat_clever_tagger.qlearning.pkl...done
Simulating timestep 300/300 - population: 26 Processing Q-Learning data...done
Saving Q-Learning data...done
Q Learning training data found, loading from somewhat_clever_tagger.qlearning.pkl...done
Simulating timestep 300/300 - population: 26 Processing Q-Learning data...done
Saving Q-Learning data...done
Q Learning training data found, loading from somewhat_clever_tagger.qlearning.pkl...done
Simulating timestep 300/300 - population: 26 Processing Q-Learning data...done
Saving Q-Learning data...done
Q Learning training data found, loading from somewhat_clever_tagger.qlearning.pkl...done
Simulating timestep 300/300 - population: 26 Processing Q-Learning data...done
Saving Q-Learning data...done
Q Learning